API 키 로딩

In [1]:
import os # 운영체제와 상호작용하기 위한 os 모듈을 가져옵니다.
from dotenv import load_dotenv # .env 파일에서 환경 변수를 로드하기 위한 dotenv 모듈의 load_dotenv 함수를 가져옵니다.
load_dotenv('..\\.env',override = True) # '.env' 파일을 로드하며, 기존 환경 변수를 덮어씁니다. 00_dev_edu\.env

# Azure OpenAI API 관련 설정을 정의합니다.
aoai_api_base = os.getenv('AZURE_OPENAI_ENDPOINT')
aoai_api_key = os.getenv('AZURE_OPENAI_API_KEY')
aoai_deployment_name = os.getenv('AZURE_OPENAI_CHAT_DEPLOYMENT_NAME_GPT4O')
aoai_api_version = os.getenv('AZURE_OPENAI_API_VERSION')
openai_api_version = os.getenv('OPENAI_API_VERSION')

doc_intelligence_endpoint = os.getenv('AZURE_DOCUMENT_INTELLIGENCE_ENDPOINT')
doc_intelligence_api_key = os.getenv('AZURE_DOCUMENT_INTELLIGENCE_KEY')
doc_intelligence_api_version = os.getenv('AZURE_DOCUMENT_INTELLIGENCE_API_VERSION')

print(f'AZURE_OPENAI_ENDPOINT: {aoai_api_base}')
print(f'AZURE_OPENAI_API_KEY: {aoai_api_key}')
print(f'AZURE_OPENAI_CHAT_DEPLOYMENT_NAME_GPT4O: {aoai_deployment_name}')
print(f'AZURE_OPENAI_API_VERSION: {aoai_api_version}')
print(f'OPENAI_API_VERSION: {openai_api_version}')
print(f'AZURE_DOCUMENT_INTELLIGENCE_ENDPOINT: {doc_intelligence_endpoint}')
print(f'AZURE_DOCUMENT_INTELLIGENCE_KEY: {doc_intelligence_api_key}')
print(f'AZURE_DOCUMENT_INTELLIGENCE_API_VERSION: {doc_intelligence_api_version}')

AZURE_OPENAI_ENDPOINT: https://aoai-spn-estus2.openai.azure.com/
AZURE_OPENAI_API_KEY: 9f84277eccac458186c0f6a50e508223
AZURE_OPENAI_CHAT_DEPLOYMENT_NAME_GPT4O: gpt-4o
AZURE_OPENAI_API_VERSION: 2024-06-01
OPENAI_API_VERSION: 2024-06-01
AZURE_DOCUMENT_INTELLIGENCE_ENDPOINT: https://docu-intellegence-estus.cognitiveservices.azure.com/
AZURE_DOCUMENT_INTELLIGENCE_KEY: a83db99ae0364c4fb473fe242b2cf7c0
AZURE_DOCUMENT_INTELLIGENCE_API_VERSION: 2024-07-31-preview


RAG를 수행할 문서 경로와 언어 설정

In [2]:
_SAMPLE = '[HIF 월간 산업이슈] 24년 10월호.pdf' # RAG를 수행할 문서 설정
_index_name: str = "idx120301" # 생성할 azure ai search 인덱스 이름 설정

_filePath = os.path.join("data", _SAMPLE) # 문서 경로 설정
print(_filePath)

_language = "Korean" # 주요 언어 설정

_file_name = _SAMPLE

data\[HIF 월간 산업이슈] 24년 10월호.pdf


모듈 import

In [3]:
from azure.ai.documentintelligence import DocumentIntelligenceClient
from azure.core.credentials import AzureKeyCredential
from azure.ai.documentintelligence.models import DocumentAnalysisFeature # Azure 문서 인텔리전스 분석 기능을 임포트합니다.
from langchain_community.document_loaders.blob_loaders import Blob # Blob 로더를 가져옵니다.
from azure.ai.documentintelligence.models import AnalyzeResult


azure document intelligence를 호출하여 문서 레이아웃 분석 실행

In [4]:
kwargs = {}
kwargs["api_version"] = doc_intelligence_api_version

docIntelligenceClient = DocumentIntelligenceClient(
    endpoint=doc_intelligence_endpoint,
    credential=AzureKeyCredential(doc_intelligence_api_key),
    headers={"x-ms-useragent": "langchain-parser/1.0.0"},
    features=[DocumentAnalysisFeature.OCR_HIGH_RESOLUTION], # 고해상도 이미지 OCR 옵션 - 추가 비용이 든다.
    **kwargs,
)

api_model="prebuilt-layout" # 레이아웃 분석 모델, 일반적인 모델 사용
contenttype="application/octet-stream"
output_contentformat="markdown" # 레이아웃 분석 결과에서 페이지 내용을 markdown 형식으로 반환

blob = Blob.from_path(_filePath)
with blob.as_bytes_io() as file_obj:
    poller = docIntelligenceClient.begin_analyze_document(
        model_id = api_model,
        analyze_request=file_obj,
        content_type = contenttype,
        output_content_format = output_contentformat,
    )

# 작업 완료 대기 및 결과 받기
try:
    docAnalyzeResult: AnalyzeResult = poller.result()  # 작업이 완료될 때까지 대기
except Exception as e:
    raise ValueError(f"문서 분석 중 오류가 발생했습니다: {e}")

레이아웃 분석 결과에 대한 serialize 테스트 및 레이아웃 분석 정보 생성
- analyze_result

In [5]:
import json

json_obj = docAnalyzeResult.as_dict() # 분석 결과를 json으로 변환
print(type(json_obj))

json_str = json.dumps(json_obj, ensure_ascii=False) # json을 string으로 변환
print(type(json_str))

con_json_obj = json.loads(json_str) # string을 다시 json으로 변환
print(type(con_json_obj))

analyze_result = AnalyzeResult(con_json_obj) # json 객체를 다시 azure document intelligence 객체로 변환
# analyze_result

<class 'dict'>
<class 'str'>
<class 'dict'>


각 페이지 번호와 페이지 텍스트를 라인 기준으로 추출

In [25]:
def extract_page_text(analyze_result: AnalyzeResult):
    page_contents = dict()
    for page in analyze_result.pages:
        page_content = ""

        for lineElement in page.lines:
            page_content += f"{lineElement.content}\n"
    
        page_contents[page.page_number] = page_content #페이지 번호 위치에 페이지 텍스트 저장

    return page_contents

page_content_list = extract_page_text(analyze_result)
page_content_list

{1: 'HIF 월간 산업 이슈\nMonthly Industrial Issue.\n2024년 8월 30일 제39호\n연\n구 원이\n예\n린\n연구 원 김종 현\n연구 위 원\n김\n문\n태\n하나로여겨된\n모두의ㄱㅇ.\nHIF 월간 산업 이슈(10월)\nMonthly Industrial Issue\n산업별 주요 이슈\n철강\n철강업, 원가 부담 가중되고 업황 정체에 따른 실적 회복 시기 지연\n• 건설업 등 전방 수요 부진이 지속되는 가운데, 중국 경기 침체에 따른 밀어내기 수출 등으로 철강업\n불황이 장기화되며 철강사들의 3분기 실적도 저하된 것으로 파악\n• 산업용 전기료 상승으로 철강사의 비용 부담이 증가하고 중국 정부의 경기 부양책에도 전방 산업\n회복이 지연되며 \'25년 철강사의 실적 개선폭은 제한적일 전망\n조선\n3Q 양호한 성장세, 그러나 중국 생산능력 확대에 따른 영향 우려\n• \'24년 대량 수주에 의한 기저효과로 인해 내년 신조 수주는 둔화될 것으로 예상되는 가운데 중국\n조선소의 증설로 글로벌 수주 경쟁이 심화되며 선가가 하락할 전망\n• 선가 하락으로 인해 \'27년 이후 인도될 물량의 수익성 악화가 예상되며, 증설을 바탕으로 한 중국\n조선소의 가격 우위 전략으로 인해 국내 조선소의 영업환경이 악화될 우려\n유통\n"불황기, 고객은 떠나도 팬은 떠나지 않는다."\', 팬덤 경제 확산과 영향\n• \'팬덤 경제\'는 특정 대상을 향한 유대감과 열정이 창출하는 경제적 가치를 의미하며, 최근 디지털\n플랫폼 및 인프라 발달, 가치 소비 확산, 모바일 보편화, 팬덤의 집단적 소비 행태 등에 따라 확대\n• 팬덤 경제 기반의 커머스 플랫폼은 팬덤 플랫폼, 라이브커머스, 커뮤니티형 커머스 등이 존재하며, 팬덤\n경제는 경기에 비교적 둔감한 특성을 지녀 소비 위축 시기에 새로운 성장 기회를 제공할 것으로 기대\nㅎ 하나은행 하나금융연구소\n',
 2: '산업 이슈\n철강\n철강업, 원가 부담 가중되고 업황 정체에 따른 실적 회복 시기 지연\n

각 페이지 번호와 페이지 텍스트를 페이지 위치 기준으로 추출 - 페이지 내용을 통으로 추출하기 때문에 markdown 구조를 유지하면서 추출한다.
- page_content_list

In [26]:
def extract_page_markdown(analyze_result: AnalyzeResult):
    page_contents = dict()
    for page in analyze_result.pages:

        content = analyze_result.content[page.spans[0]['offset']: page.spans[0]['offset'] + page.spans[0]['length']]
    
        page_contents[page.page_number] = content

    return page_contents

page_content_list = extract_page_markdown(analyze_result)
page_content_list

{1: '<!-- PageHeader="HIF 월간 산업 이슈 Monthly Industrial Issue." -->\n\n2024년 8월 30일 제39호\n\n연\n구 원이\n예\n린\n연구 원 김종 현\n\n연구 위 원\n김\n문\n태\n\n<!-- PageHeader="하나로여겨된 모두의ㄱㅇ." -->\n\n\n# HIF 월간 산업 이슈(10월) Monthly Industrial Issue\n\n\n# 산업별 주요 이슈\n\n\n## 철강 철강업, 원가 부담 가중되고 업황 정체에 따른 실적 회복 시기 지연\n\n• 건설업 등 전방 수요 부진이 지속되는 가운데, 중국 경기 침체에 따른 밀어내기 수출 등으로 철강업\n불황이 장기화되며 철강사들의 3분기 실적도 저하된 것으로 파악\n\n• 산업용 전기료 상승으로 철강사의 비용 부담이 증가하고 중국 정부의 경기 부양책에도 전방 산업\n회복이 지연되며 \'25년 철강사의 실적 개선폭은 제한적일 전망\n\n\n## 조선 3Q 양호한 성장세, 그러나 중국 생산능력 확대에 따른 영향 우려\n\n• \'24년 대량 수주에 의한 기저효과로 인해 내년 신조 수주는 둔화될 것으로 예상되는 가운데 중국\n조선소의 증설로 글로벌 수주 경쟁이 심화되며 선가가 하락할 전망\n\n• 선가 하락으로 인해 \'27년 이후 인도될 물량의 수익성 악화가 예상되며, 증설을 바탕으로 한 중국\n조선소의 가격 우위 전략으로 인해 국내 조선소의 영업환경이 악화될 우려\n\n\n## 유통 "불황기, 고객은 떠나도 팬은 떠나지 않는다."\', 팬덤 경제 확산과 영향\n\n• \'팬덤 경제\'는 특정 대상을 향한 유대감과 열정이 창출하는 경제적 가치를 의미하며, 최근 디지털\n플랫폼 및 인프라 발달, 가치 소비 확산, 모바일 보편화, 팬덤의 집단적 소비 행태 등에 따라 확대\n\n• 팬덤 경제 기반의 커머스 플랫폼은 팬덤 플랫폼, 라이브커머스, 커뮤니티형 커머스 등이 존재하며, 팬덤\n경제는 경기에 비교적 둔감한 특성을 지녀 소비 위축 시기에 새로운 성장

LLM 모델 호출 함수 정의
- llm_gpt4o_mini
- llm_gpt4o

In [27]:
from langchain_openai import AzureChatOpenAI

def llm_gpt4o_mini(temperature=0.0, streaming=False):
    # os.environ['OPENAI_API_VERSION'] = os.getenv('AZURE_OPENAI_API_VERSION')
    # print(f"AZURE_OPENAI_API_VERSION: {os.getenv('AZURE_OPENAI_API_VERSION')}")
    # print(f"OPENAI_API_VERSION: {os.getenv('OPENAI_API_VERSION')}") #OPENAI_API_VERSION
    
    llm = AzureChatOpenAI(
        api_key = os.getenv('AZURE_OPENAI_API_KEY'), # Azure OpenAI API 키를 환경 변수에서 가져옵니다.
        api_version = os.getenv('AZURE_OPENAI_API_VERSION'), # OpenAI API 버전을 설정합니다.
        azure_endpoint = os.getenv('AZURE_OPENAI_ENDPOINT'), # Azure OpenAI 엔드포인트를 환경 변수에서 가져옵니다.
        model= os.getenv('AZURE_OPENAI_CHAT_DEPLOYMENT_NAME_GPT4MINI'), # 사용할 모델을 설정합니다.
        streaming=streaming, # 스트리밍
        temperature=temperature,
    )    
    return llm

def llm_gpt4o(temperature=0.0, streaming=False):

    # print(f"AZURE_OPENAI_API_VERSION: {os.getenv('AZURE_OPENAI_API_VERSION')}")
    # print(f"OPENAI_API_VERSION: {os.getenv('OPENAI_API_VERSION')}") #OPENAI_API_VERSION

    llm = AzureChatOpenAI(
        api_key = os.getenv('AZURE_OPENAI_API_KEY'), # Azure OpenAI API 키를 환경 변수에서 가져옵니다.
        api_version = os.getenv('AZURE_OPENAI_API_VERSION'), # OpenAI API 버전을 설정합니다.
        azure_endpoint = os.getenv('AZURE_OPENAI_ENDPOINT'), # Azure OpenAI 엔드포인트를 환경 변수에서 가져옵니다.
        model= os.getenv('AZURE_OPENAI_CHAT_DEPLOYMENT_NAME_GPT4O'), # 사용할 모델을 설정합니다.
        streaming=streaming, # 스트리밍
        temperature=temperature,
    )
    return llm

페이지별 요약 정보 생성
- summary_list

In [28]:
from langchain_core.prompts import PromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.documents import Document # LangChain 코어의 Document 클래스를 가져옵니다.

"""
문장을 아래의 요청에 따라 요약해 주십시오.

요청:

1. 주요 내용을 bullet point로 요약하십시오.
2. 요약은 본문의 언어와 동일하게 작성하십시오.
3. 기술 용어는 번역하지 마십시오.
4. 불필요한 정보를 포함하지 마십시오.
5. 요약에는 중요한 엔터티와 수치 정보를 포함해야 합니다.
본문: {context}

요약:
"""

# 문서 요약 체인 생성
def create_text_summary_chain():
    # 요약을 위한 프롬프트 템플릿을 정의합니다.
    prompt = PromptTemplate.from_template(
        """Please summarize the sentence according to the following REQUEST.
        
    REQUEST:
    1. Summarize the main points in bullet points.
    2. Write the summary in same language as the context.
    3. DO NOT translate any technical terms.
    4. DO NOT include any unnecessary information.
    5. Summary must include important entities, numerical values.

    CONTEXT:
    {context}

    SUMMARY:"
    """
    )

    # ChatOpenAI 모델의 또 다른 인스턴스를 생성합니다. (이전 인스턴스와 동일한 설정)
    llm = llm_gpt4o()

    # 문서 요약을 위한 체인을 생성합니다.
    # 이 체인은 여러 문서를 입력받아 하나의 요약된 텍스트로 결합합니다.
    text_summary_chain = create_stuff_documents_chain(llm, prompt)

    return text_summary_chain

# 페이지별 문서 요약 생성
def create_page_summary(page_contents):
    content_summary = dict()

    inputs = [
        {"context": [Document(page_content=text)]}
        for page_num, text in page_contents.items()
    ]

    # 요약 체인 생성
    content_summary_chain = create_text_summary_chain()

    # text_summary_chain을 사용하여 일괄 처리로 요약을 생성합니다.
    summaries = content_summary_chain.batch(inputs)

    # 생성된 요약을 페이지 번호와 함께 딕셔너리에 저장합니다.
    for page_num, summary in enumerate(summaries):
        content_summary[page_num+1] = summary

    return content_summary

summary_list = create_page_summary(page_content_list)
summary_list

{1: "- 철강업:\n  - 전방 수요 부진과 중국 경기 침체로 철강업 불황 장기화\n  - 3분기 실적 저하, 산업용 전기료 상승으로 비용 부담 증가\n  - '25년 실적 개선폭 제한적 전망\n\n- 조선업:\n  - 3분기 양호한 성장세, 그러나 중국 조선소 증설로 글로벌 수주 경쟁 심화\n  - 선가 하락으로 '27년 이후 인도 물량 수익성 악화 우려\n  - 중국 조선소의 가격 우위 전략으로 국내 조선소 영업환경 악화 가능성\n\n- 유통업:\n  - '팬덤 경제' 확산, 디지털 플랫폼 및 가치 소비 확산에 기인\n  - 팬덤 경제 기반 커머스 플랫폼 존재, 경기 둔감 특성으로 소비 위축 시 성장 기회 제공 기대",
 2: '- 철강업: 원가 부담 증가, 업황 정체로 실적 회복 시기 지연\n- 조선업: 3분기 양호한 성장세, 중국 생산능력 확대 영향 우려\n- 유통업: 불황기에도 팬덤 경제 확산 및 영향',
 3: "- 철강업의 불황이 장기화되며 3분기 실적 저하 예상\n  - 건설 경기 침체와 중국발 철강 공급 과잉 지속\n  - '24.3Q 중국 철강 수입량 전년 동기 대비 4.8% 증가\n  - 국내 철강 가격 약세 지속\n\n- 주요 철강사들의 3분기 실적 부진\n  - 포스코, 현대제철 등 4개사 매출액 전년 동기 대비 7.4% 하락\n  - 합산 영업이익률 3%p 하락하여 3.4%로 추정\n  - 포스코홀딩스 영업이익 전년 동기 대비 45.5% 하락\n  - 현대제철 영업이익 전년 동기 대비 64% 하락\n\n- 중국의 밀어내기 수출과 경기부양책 영향\n  - 철광석 등 원자재 가격 상승\n  - 산업용 전기 요금 인상으로 철강사 재무 부담 확대\n\n- '25년 철강사 실적 개선폭 제한적 전망",
 4: "- 10월 24일부터 산업용 전기 평균 요금이 9.7% 인상되어 철강업계의 원가 부담이 증가.\n  - 대기업 기준 전기요금이 16.9/1kWh 인상, 철강업의 원가 부담 약 3,400억원 증가 예상.\n  \n- 4분기 전기료 인상으

문서에서 이미지를 추출하여 이미지 파일로 저장

In [30]:
import fitz  # PyMuPDF # PyMuPDF 모듈을 가져옵니다. PDF 처리에 사용됩니다.
import mimetypes # MIME 타입을 추측하기 위한 모듈을 가져옵니다.
from mimetypes import guess_type # 파일의 MIME 타입을 추측하는 함수를 가져옵니다.
from PIL import Image # 이미지 처리를 위한 PIL 모듈을 가져옵니다.


# 이미지 출력 폴더 생성
def ensure_output_folder(output_folder):
    """
    출력 폴더가 존재하지 않으면 생성하고, 쓰기 권한을 확인합니다.

    Args:
        output_folder (str): 출력 폴더 경로.

    Raises:
        PermissionError: 폴더에 대한 쓰기 권한이 없는 경우.
    """
    if not os.path.exists(output_folder):
        try:
            os.makedirs(output_folder, exist_ok=True)
            print(f"Created output folder: {output_folder}")
        except Exception as e:
            print(f"Error creating output folder {output_folder}: {e}")
            raise
    if not os.access(output_folder, os.W_OK):
        raise PermissionError(f"No write permission for the output folder: {output_folder}")
    
def crop_image_from_image(image_path, page_number, bounding_box):
    """
    Crops an image based on a bounding box.

    :param image_path: Path to the image file.
    :param page_number: The page number of the image to crop (for TIFF format).
    :param bounding_box: A tuple of (left, upper, right, lower) coordinates for the bounding box.
    :return: A cropped image.
    :rtype: PIL.Image.Image
    """
    with Image.open(image_path) as img:
        if img.format == "TIFF":
            # Open the TIFF image
            img.seek(page_number)
            img = img.copy()
            
        # The bounding box is expected to be in the format (left, upper, right, lower).
        cropped_image = img.crop(bounding_box)
        return cropped_image
    
def crop_image_from_pdf_page(pdf_path, page_number, bounding_box):
    """
    Crops a region from a given page in a PDF and returns it as an image.

    :param pdf_path: Path to the PDF file.
    :param page_number: The page number to crop from (0-indexed).
    :param bounding_box: A tuple of (x0, y0, x1, y1) coordinates for the bounding box.
    :return: A PIL Image of the cropped area.
    """
    doc = fitz.open(pdf_path)
    page = doc.load_page(page_number)
    
    # Cropping the page. The rect requires the coordinates in the format (x0, y0, x1, y1).
    # PDF 좌표계에서 1포인트는 1/72인치로 정의됩니다. 
    # 따라서, PDF의 경계 상자 좌표는 72 DPI(Dots Per Inch) 해상도를 기준으로 합니다.
    bbx = [x * 72 for x in bounding_box]
    rect = fitz.Rect(bbx)

    # 이미지를 처리할 때는 더 높은 해상도, 예를 들어 300 DPI를 사용하는 경우가 많습니다. 
    # 이러한 해상도 차이를 보정하기 위해 경계 상자의 좌표를 변환해야 합니다.
    # 예를 들어, 300 DPI 해상도로 이미지를 렌더링할 경우, 좌표를 변환하기 위해 300/72, 즉 약 4.17을 곱합니다. 
    # 이렇게 하면 PDF 좌표계의 포인트 단위가 이미지의 픽셀 단위와 정확히 일치하게 되어, 경계 상자가 올바른 위치와 크기로 적용됩니다.
    pix = page.get_pixmap(matrix=fitz.Matrix(300/72, 300/72), clip=rect)
    
    img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
    
    doc.close()

    return img

def crop_image_from_file(file_path, page_number, bounding_box):
    """
    Crop an image from a file.

    Args:
        file_path (str): The path to the file.
        page_number (int): The page number (for PDF and TIFF files, 0-indexed).
        bounding_box (tuple): The bounding box coordinates in the format (x0, y0, x1, y1).

    Returns:
        A PIL Image of the cropped area.
    """
    mime_type = mimetypes.guess_type(file_path)[0]
    
    if mime_type == "application/pdf":
        return crop_image_from_pdf_page(file_path, page_number, bounding_box)
    else:
        return crop_image_from_image(file_path, page_number, bounding_box)
    

def image_cropp(image_region, image_idx, input_file_path, output_folder, file_name):
    boundingbox = (
        image_region.polygon[0],  # x0 (left)
        image_region.polygon[1],  # y0 (top)
        image_region.polygon[4],  # x1 (right)
        image_region.polygon[5]   # y1 (bottom)
    )
    cropped_image = crop_image_from_file(input_file_path, image_region.page_number - 1, boundingbox) # page_number is 1-indexed
    
    # Get the base name of the file
    base_name = os.path.basename(input_file_path)
    # Remove the file extension
    file_name_without_extension = os.path.splitext(base_name)[0]
    
    output_file = f"{file_name_without_extension}_cropped_{file_name}_{image_idx}.png"
    # output_file = f"{file_name_without_extension}_cropped_image_{image_idx}.png"
    cropped_image_filename = os.path.join(output_folder, output_file)
    cropped_image.save(cropped_image_filename)
    return cropped_image_filename


#이미지 자르고 저장, 이미지 경로, 이미지 캡션 정보, 이미지가 속한 페이지 정보 저장
def create_image_cropp_info(analyze_result_info: AnalyzeResult, input_file_path, output_folder):
    image_cropped_info = []
    ensure_output_folder(output_folder)
    for idx, figure in enumerate(analyze_result_info.figures):
        caption_content = ""
        page_num = 0
        if figure.caption: # 이미지의 캡션 정보가 있으면 추출
            caption_content = figure.caption.content
            caption_region = figure.caption.bounding_regions
            
            for region in figure.bounding_regions:
                if region not in caption_region and len(region.polygon) >= 6: # 너무 작은 이미지는 제외
                    page_num = region.page_number                    
                    cropped_image_filename = image_cropp(region, idx, input_file_path, output_folder, "image")                    
                    print(f"cropped_image_filename: {cropped_image_filename}")
        else:
            for region in figure.bounding_regions:
                if len(region.polygon) >= 6:
                    page_num = region.page_number
                    cropped_image_filename = image_cropp(region, idx, input_file_path, output_folder, "image")
                    print(f"cropped_image_filename: {cropped_image_filename}")

        image_data = {
            "id": idx, # 이미지 인덱스
            "page_number": page_num, # 이미지가 속한 페이지 번호
            "image": cropped_image_filename, # 이미지 경로와 파일명
            "caption": caption_content # 이미지 캡션
        }
        image_cropped_info.append(image_data)
    return image_cropped_info

문서에서 추출한 이미지를 'data\image_cropped' 폴더에 저장 (차트 등의 표를 제외한 모든 이미지)
- image_cropped_list

In [31]:
outputfolder = os.path.join("data", "image_cropped")
image_cropped_list = create_image_cropp_info(analyze_result, _filePath, outputfolder)
image_cropped_list

Created output folder: data\image_cropped
cropped_image_filename: data\image_cropped\[HIF 월간 산업이슈] 24년 10월호_cropped_image_0.png
cropped_image_filename: data\image_cropped\[HIF 월간 산업이슈] 24년 10월호_cropped_image_1.png
cropped_image_filename: data\image_cropped\[HIF 월간 산업이슈] 24년 10월호_cropped_image_2.png
cropped_image_filename: data\image_cropped\[HIF 월간 산업이슈] 24년 10월호_cropped_image_3.png
cropped_image_filename: data\image_cropped\[HIF 월간 산업이슈] 24년 10월호_cropped_image_4.png
cropped_image_filename: data\image_cropped\[HIF 월간 산업이슈] 24년 10월호_cropped_image_5.png
cropped_image_filename: data\image_cropped\[HIF 월간 산업이슈] 24년 10월호_cropped_image_6.png
cropped_image_filename: data\image_cropped\[HIF 월간 산업이슈] 24년 10월호_cropped_image_7.png
cropped_image_filename: data\image_cropped\[HIF 월간 산업이슈] 24년 10월호_cropped_image_8.png
cropped_image_filename: data\image_cropped\[HIF 월간 산업이슈] 24년 10월호_cropped_image_9.png
cropped_image_filename: data\image_cropped\[HIF 월간 산업이슈] 24년 10월호_cropped_image_10.png
cropped_ima

[{'id': 0,
  'page_number': 1,
  'image': 'data\\image_cropped\\[HIF 월간 산업이슈] 24년 10월호_cropped_image_0.png',
  'caption': ''},
 {'id': 1,
  'page_number': 3,
  'image': 'data\\image_cropped\\[HIF 월간 산업이슈] 24년 10월호_cropped_image_1.png',
  'caption': '그림1 | 글로벌 철강 수요 추이 및 전망'},
 {'id': 2,
  'page_number': 3,
  'image': 'data\\image_cropped\\[HIF 월간 산업이슈] 24년 10월호_cropped_image_2.png',
  'caption': '그림2 | 주요 철강 3개사의 3분기 실적'},
 {'id': 3,
  'page_number': 4,
  'image': 'data\\image_cropped\\[HIF 월간 산업이슈] 24년 10월호_cropped_image_3.png',
  'caption': '그림3 | 철근 기준가격 및 유통가격 추이'},
 {'id': 4,
  'page_number': 4,
  'image': 'data\\image_cropped\\[HIF 월간 산업이슈] 24년 10월호_cropped_image_4.png',
  'caption': '그림4 | 국내 철강업체 실적 전망'},
 {'id': 5,
  'page_number': 5,
  'image': 'data\\image_cropped\\[HIF 월간 산업이슈] 24년 10월호_cropped_image_5.png',
  'caption': '그림5 | 글로벌 신조선 수주량 추이'},
 {'id': 6,
  'page_number': 5,
  'image': 'data\\image_cropped\\[HIF 월간 산업이슈] 24년 10월호_cropped_image_6.png',
  'caption': '그림6 | 국

테이블 이미지를 추출하여 파일로 저장

In [32]:
#테이블 이미지 자르고 저장, 이미지 경로, 테이블 캡션 정보, 테이블이 속한 페이지 정보 저장
def create_table_cropp_info(analyze_result_info: AnalyzeResult, input_file_path, output_folder):
    table_cropped_info = []
    ensure_output_folder(output_folder)
    for idx, table in enumerate(analyze_result_info.tables):
        caption_content = ""
        page_num = 0
        if table.caption: # 테이블 캡션이 있으면 추출
            caption_content = table.caption.content
            caption_region = table.caption.bounding_regions
            
            for region in table.bounding_regions:
                if region not in caption_region and len(region.polygon) >= 6: # 너무 작은 이미지는 제외
                    page_num = region.page_number                    
                    cropped_image_filename = image_cropp(region, idx, input_file_path, output_folder, "table")                    
                    print(f"cropped_image_filename: {cropped_image_filename}")
        else:
            for region in table.bounding_regions:
                if len(region.polygon) >= 6:
                    page_num = region.page_number
                    cropped_image_filename = image_cropp(region, idx, input_file_path, output_folder, "table")
                    print(f"cropped_image_filename: {cropped_image_filename}")
        
        table_markdown = analyze_result_info.content[table.spans[0]['offset']: table.spans[0]['offset']+table.spans[0]['length']]

        table_data = {
            "id": idx, # 테이블 이미지 인덱스
            "page_number": page_num, # 테이블이 속한 페이지 번호
            "image": cropped_image_filename, # 테이블 이미지 경로와 파일명
            "caption": caption_content, # 테이블 캡션
            "markdown": table_markdown # markdown 으로 작성된 테이블 내용 
        }
        table_cropped_info.append(table_data)
    return table_cropped_info

문서에서 추출한 테이블 이미지를 'data\image_cropped' 폴더에 저장 (모든 테이블 이미지)
- table_cropped_list

In [33]:
table_cropped_list = create_table_cropp_info(analyze_result, _filePath, outputfolder)
table_cropped_list

cropped_image_filename: data\image_cropped\[HIF 월간 산업이슈] 24년 10월호_cropped_table_0.png
cropped_image_filename: data\image_cropped\[HIF 월간 산업이슈] 24년 10월호_cropped_table_1.png
cropped_image_filename: data\image_cropped\[HIF 월간 산업이슈] 24년 10월호_cropped_table_2.png


[{'id': 0,
  'page_number': 6,
  'image': 'data\\image_cropped\\[HIF 월간 산업이슈] 24년 10월호_cropped_table_0.png',
  'caption': '표1 | 중국 조선소의 증설 동향',
  'markdown': '<table>\n<caption>표1 | 중국 조선소의 증설 동향</caption>\n<tr>\n<th>조선소</th>\n<th>가동 예상일</th>\n<th>생산규모 (CGT)</th>\n<th>주요 선종</th>\n</tr>\n<tr>\n<td>Hengli SB</td>\n<td>2026</td>\n<td>831,000</td>\n<td>벌크, 탱커</td>\n</tr>\n<tr>\n<td>Wison Qidong</td>\n<td>2026</td>\n<td>-</td>\n<td>해양플랜트</td>\n</tr>\n<tr>\n<td>YangzijIang Holdings</td>\n<td>2026</td>\n<td>-</td>\n<td>LNG선</td>\n</tr>\n<tr>\n<td>New Times SB</td>\n<td>2026</td>\n<td>882,00</td>\n<td>탱커, 컨테이너</td>\n</tr>\n<tr>\n<td>Hantong WinG HI</td>\n<td>2026</td>\n<td>37,000</td>\n<td>Ultramaxes</td>\n</tr>\n<tr>\n<td>CMCS</td>\n<td>2026</td>\n<td>-</td>\n<td>크루즈, 여객선</td>\n</tr>\n<tr>\n<td>Shanghai Waigaoqiao</td>\n<td>2025</td>\n<td>973,000</td>\n<td>컨테이너, 탱커</td>\n</tr>\n<tr>\n<td>COSCO HI (Zhoushan)</td>\n<td>2026</td>\n<td>384,000</td>\n<td>벌크</td>\n</tr>\n<tr>\n<td>Chizhou Guichi</t

이미지 요약정보 생성 함수

In [34]:
from langchain_core.runnables import chain
from cls_multi_modal import MultiModal

# 프롬프트와 체인을 생성하여 이미지 요약 정보 생성
"""
이미지에서 유용한 정보를 추출하는 전문가로서, 주어진 이미지를 분석하여 핵심 요소를 식별하고 요약합니다. 
이를 통해 추후 검색이나 활용이 가능하도록 정보를 정리합니다.
숫자 데이터가 포함된 경우, 중요한 통찰을 도출합니다. 
또한, 해당 이미지와 관련하여 사용자가 질문할 수 있는 다섯 가지 가상의 질문을 제공합니다.


Output must be written in {language}.
Please provide the output in {language} without any additional commentary or annotations.
"""
@chain
def extract_image_summary(data_batches):
    # 객체 생성
    llm = llm_gpt4o()

    system_prompt = """You are an expert in extracting useful information from IMAGE.
With a given image, your task is to extract key entities, summarize them, and write useful information that can be used later for retrieval.
If the numbers are present, summarize important insights from the numbers.
Also, provide five hypothetical questions based on the image that users can ask.
"""

    image_paths = []
    system_prompts = []
    user_prompts = []

    for data_batch in data_batches:
        context = data_batch["text"] + "\n\n<caption>" + data_batch["caption"] + "</caption>"
        image_path = data_batch["image"]
        language = data_batch["language"]
        user_prompt_template = f"""Here is the context related to the image: {context}
        
###

Output Format:

<image>
<title>
[title]
</title>
<summary>
[summary]
</summary>
<entities> 
[entities]
</entities>
<data_insights>
[data_insights]
</data_insights>
<hypothetical_questions>
[hypothetical_questions]
</hypothetical_questions>
</image>

Please provide the output in {language} without any additional commentary or annotations.
"""
        image_paths.append(image_path)
        system_prompts.append(system_prompt)
        user_prompts.append(user_prompt_template)

    # 멀티모달 객체 생성
    multimodal_llm = MultiModal(llm)

    # 이미지 파일로 부터 질의
    answer = multimodal_llm.batch(
        image_paths, system_prompts, user_prompts, display_image=False
    )
    return answer


# 프롬프트에 삽입할 이미지 정보 생성
def create_image_summary_data_batches(pagecontentlist, summaryList, imageCroppedList):
    data_batchs=[]
    for page_num in summaryList.keys():
        
        imageinfo_page_items = [imageinfo for imageinfo in imageCroppedList if imageinfo["page_number"] == page_num]
        for imageItem in imageinfo_page_items:
            #print(f"page: {page_num}, image item: {imageItem["image"]}")
            data_batchs.append(
                {
                    "image": imageItem["image"],
                    "text": pagecontentlist[page_num], # 페이지 전체 내용
                    # "text": summaryList[page_num], # 페이지 요약 내용
                    "page": page_num,
                    "id": imageItem["id"],
                    "caption": imageItem["caption"],
                    "language": _language,
                }
            )
    return data_batchs

# 이미지별 요약 정보 생성 
def create_image_summary(pageocntentlist, summaryList, imageCroppedList):
    image_summary_data_batches = create_image_summary_data_batches(pageocntentlist, summaryList, imageCroppedList)

    image_summaries = extract_image_summary.invoke(
        image_summary_data_batches,
    )

    image_summary_output = dict()

    for data_batch, image_summary in zip(
        image_summary_data_batches, image_summaries
    ):
        image_summary_output[data_batch["id"]] = image_summary
    
    return image_summary_output

이미지 요약정보 생성
- image_summary_data_list

In [35]:
image_summary_data_list = create_image_summary(page_content_list, summary_list, image_cropped_list)
image_summary_data_list

{0: '<image>\n<title>\nHIF 월간 산업 이슈(10월)\n</title>\n<summary>\n2024년 8월 30일 발행된 제39호 HIF 월간 산업 이슈에서는 철강, 조선, 유통 산업의 주요 이슈를 다루고 있다. 철강업은 원가 부담 증가와 업황 정체로 실적 회복이 지연되고 있으며, 조선업은 중국의 생산능력 확대에 따른 글로벌 경쟁 심화로 선가 하락이 예상된다. 유통업에서는 팬덤 경제의 확산이 새로운 성장 기회를 제공할 것으로 기대된다.\n</summary>\n<entities> \n철강업, 조선업, 유통업, 중국, 팬덤 경제\n</entities>\n<data_insights>\n- 철강업은 전방 수요 부진과 중국 경기 침체로 인해 불황이 장기화되고 있으며, 2025년 실적 개선폭이 제한적일 것으로 전망된다.\n- 조선업은 2024년 대량 수주에 따른 기저효과로 신조 수주가 둔화될 것으로 예상되며, 중국 조선소의 증설로 인해 글로벌 수주 경쟁이 심화될 전망이다.\n- 유통업에서는 팬덤 경제가 경기에 둔감한 특성을 지녀 소비 위축 시기에 새로운 성장 기회를 제공할 것으로 기대된다.\n</data_insights>\n<hypothetical_questions>\n1. 철강업의 원가 부담 증가 요인은 무엇인가요?\n2. 중국 조선소의 증설이 글로벌 시장에 미치는 영향은 무엇인가요?\n3. 팬덤 경제가 유통업에 미치는 긍정적인 영향은 무엇인가요?\n4. 2025년 철강업의 실적 개선이 제한적인 이유는 무엇인가요?\n5. 팬덤 경제의 확산이 소비자 행동에 미치는 영향은 무엇인가요?\n</hypothetical_questions>\n</image>',
 1: '<image>\n<title>\n글로벌 철강 수요 추이 및 전망\n</title>\n<summary>\n글로벌 철강 수요는 2021년 2.8% 증가했으나, 2022년에는 -3.1% 감소했다. 2023년과 2024년에는 각각 -0.8%와 -0.9%로 감소세가 지속될 것으로 예상되며, 2025년에

테이블 요약정보 생성 함수

In [36]:
"""
주어진 표에서 유용한 정보를 추출하는 전문가로서, 이미지로 제공된 표를 분석하여 핵심 엔터티를 식별하고 요약합니다. 
숫자 데이터가 포함된 경우, 중요한 통찰을 도출합니다. 
또한, 사용자가 이미지 기반으로 질문할 수 있는 다섯 가지 가상의 질문을 제공합니다.

Output must be written in {language}.
"""
# 프롬프트와 체인을 생성하여 테이블 요약 정보 생성
@chain
def extract_table_summary(data_batches):
    # 객체 생성
    llm = llm_gpt4o()

    system_prompt = """You are an expert in extracting useful information from TABLE. 
With a given image, your task is to extract key entities, summarize them, and write useful information that can be used later for retrieval.
If the numbers are present, summarize important insights from the numbers.
Also, provide five hypothetical questions based on the image that users can ask.
"""

    image_paths = []
    system_prompts = []
    user_prompts = []

    for data_batch in data_batches:
        context = data_batch["text"] + "\n\n<caption>" + data_batch["caption"] + "</caption>"
        image_path = data_batch["table"]
        language = data_batch["language"]
        user_prompt_template = f"""Here is the context related to the image of table: {context}
        
###

Output Format:

<table>
<title>
[title]
</title>
<summary>
[summary]
</summary>
<entities> 
[entities]
</entities>
<data_insights>
[data_insights]
</data_insights>
<hypothetical_questions>
[hypothetical_questions]
</hypothetical_questions>
</table>

Please provide the output in {language} without any additional commentary or annotations.
"""
        image_paths.append(image_path)
        system_prompts.append(system_prompt)
        user_prompts.append(user_prompt_template)

    # 멀티모달 객체 생성
    multimodal_llm = MultiModal(llm)

    # 이미지 파일로 부터 질의
    answer = multimodal_llm.batch(
        image_paths, system_prompts, user_prompts, display_image=False
    )
    return answer


def create_table_summary_data_batches(pagecontentlist, summaryList, tableCroppedList):
    data_batchs=[]
    for page_num in summaryList.keys():
        
        tableinfo_page_items = [tableinfo for tableinfo in tableCroppedList if tableinfo["page_number"] == page_num]
        for tableItem in tableinfo_page_items:
            #print(f"page: {page_num}, image item: {imageItem["image"]}")
            data_batchs.append(
                {
                    "table": tableItem["image"],
                    "text": pagecontentlist[page_num],
                    # "text": summaryList[page_num],
                    "page": page_num,
                    "id": tableItem["id"],
                    "caption": tableItem["caption"],
                    "language": _language,
                }
            )
    return data_batchs
   

def create_table_summary(pagecontentlist, summaryList, tableCroppedList):
    table_summary_data_batches = create_table_summary_data_batches(pagecontentlist, summaryList, tableCroppedList)

    table_summaries = extract_table_summary.invoke(
        table_summary_data_batches,
    )

    table_summary_output = dict()

    for data_batch, table_summary in zip(
        table_summary_data_batches, table_summaries
    ):
        table_summary_output[data_batch["id"]] = table_summary
    
    return table_summary_output

테이블 이미지 요약정보 생성
- table_summary_data_list

In [37]:
table_summary_data_list = create_table_summary(page_content_list, summary_list, table_cropped_list)
table_summary_data_list

{0: '```html\n<table>\n<title>\n중국 조선소의 증설 동향\n</title>\n<summary>\n중국 조선소의 증설 계획에 대한 정보로, 2025년과 2026년에 걸쳐 다양한 선종의 생산이 예정되어 있다.\n</summary>\n<entities> \n조선소, 가동 예상일, 생산규모 (CGT), 주요 선종\n</entities>\n<data_insights>\n- Shanghai Waigaoqiao는 2025년에 가장 큰 생산규모인 973,000 CGT를 계획하고 있다.\n- 2026년에는 Hengli SB와 New Times SB가 각각 831,000 CGT와 882,000 CGT의 생산을 계획 중이다.\n- 주요 선종으로는 벌크, 탱커, 컨테이너, LNG선 등이 포함되어 있다.\n</data_insights>\n<hypothetical_questions>\n1. 2026년에 가동 예정인 조선소 중 가장 큰 생산규모를 가진 곳은 어디인가요?\n2. Shanghai Waigaoqiao의 주요 선종은 무엇인가요?\n3. 2025년과 2026년의 생산규모 차이는 어떻게 되나요?\n4. LNG선을 생산하는 조선소는 어디인가요?\n5. 중국 조선소의 증설이 한국 조선소에 미치는 영향은 무엇인가요?\n</hypothetical_questions>\n</table>\n```',
 1: '```markdown\n<table>\n<title>\n팬덤 경제 확산의 배경\n</title>\n<summary>\n팬덤 경제는 특정 대상을 향한 유대감과 열정이 창출하는 경제적 가치를 의미하며, 디지털 플랫폼의 성장, 인프라 발달, 모바일 사용 보편화, 소비 트렌드 변화, 팬덤의 집단적 소비 행태에 의해 확산되고 있다.\n</summary>\n<entities> \n디지털 플랫폼 성장, 인프라 발달, 모바일 사용 보편화, 소비 트렌드 변화, 팬덤의 집단적 소비 행태\n</entities>\n<data_insights>\n1. 디지털 플랫폼의 글로벌 확산은 팬덤

Image, Table 에서 추출된 데이터 Vector DB 생성을 위한 문서 생성

- Title, Summary, Entities 는 임베딩 검색에 걸리기 위한 문서로 생성
- hypothetical_questions 는 임베딩 검색에 걸리기 위한 문서로 생성

특정 요소 추출 또는 제외 함수

In [38]:
import re

# 특정 태그의 내용을 반환
def extract_tag_content(content, tag):
    pattern = rf"<{tag}>(.*?)</{tag}>"
    match = re.search(pattern, content, re.DOTALL)

    if match:
        return match.group(1).strip()
    else:
        return None

# 특정 태그를 제외한 내용을 반환
def extract_non_tag_content(content, tag):
    pattern = rf"<{tag}>.*?</{tag}>"
    result = re.sub(pattern, "", content, flags=re.DOTALL)
    return result.strip()

# 특정 태그의 내용을 반환 - 태그 포함
def extract_include_tag_content(content, tag):
    pattern = rf"(<{tag}>.*?</{tag}>)"
    match = re.search(pattern, content, re.DOTALL)

    if match:
        return match.group(1)
    else:
        return None

In [39]:
print(extract_tag_content(image_summary_data_list[9], "hypothetical_questions"))

- 팬덤 경제가 다른 산업에 미치는 영향은 무엇인가요?
- 가수 임영웅의 팬덤이 산업계에 미친 구체적인 사례는 무엇인가요?
- 디지털 플랫폼의 성장이 팬덤 경제에 어떤 영향을 미쳤나요?
- 팬덤 경제의 확산이 소비 트렌드에 어떤 변화를 가져왔나요?
- 팬덤 경제가 불황기에 어떻게 새로운 성장 기회를 제공하나요?


In [40]:
print(extract_non_tag_content(image_summary_data_list[9], "hypothetical_questions"))

```markdown
<image>
<title>
가수 임영웅 광고 및 효과
</title>
<summary>
팬덤 경제의 확산과 그 경제적 영향력을 설명하며, 가수 임영웅의 광고 효과를 통해 팬덤 경제가 산업계에 미치는 긍정적 영향을 강조한다.
</summary>
<entities> 
- 팬덤 경제
- 가수 임영웅
- G4 렉스턴
- 정관장
- 디지털 플랫폼
- 모바일 사용
</entities>
<data_insights>
- 팬덤 경제는 특정 대상에 대한 유대감과 열정이 창출하는 경제적 가치로, 디지털 플랫폼과 가치 소비의 확산에 따라 확대되고 있다.
- 가수 임영웅의 팬덤은 자동차와 건강식품 산업의 매출 증가에 기여하며, 팬덤 경제의 실질적 경제적 가치를 보여준다.
- 디지털 플랫폼의 성장, 인프라 발달, 모바일 사용의 보편화, 소비 트렌드 변화 등이 팬덤 경제 확산의 배경으로 작용하고 있다.
</data_insights>

</image>
```


테이블 데이터 정규화

In [41]:
# XML 파싱을 위해 import 문을 추가합니다
import xml.etree.ElementTree as ET

# 테이블의 요약 정보를 정규화 하여 벡터DB에 저장 
def convert_to_markdown_table(table_summary):
    html = "<table>\n"

    # table_summary가 문자열인 경우를 처리합니다
    if isinstance(table_summary, str):
        # XML 파싱을 사용하여 문자열에서 데이터를 추출합니다
        root = ET.fromstring(table_summary)
        for child in root:
            html += f"  <tr>\n    <th>{child.tag}</th>\n    <td>"

            if child.tag in ["entities", "data_insights"]:
                html += "<ul>\n"
                for item in child.text.strip().split("\n- "):
                    if item.strip():
                        html += f"      <li>{item.strip()}</li>\n"
                html += "    </ul>"
            elif child.tag == "hypothetical_questions":
                html += "<ol>\n"
                for item in child.text.strip().split("\n"):
                    if item.strip():
                        html += f"      <li>{item.strip()}</li>\n"
                html += "    </ol>"
            else:
                html += child.text.strip()

            html += "</td>\n  </tr>\n"
    else:
        # 기존의 딕셔너리 처리 로직을 유지합니다
        for key, value in table_summary.items():
            html += f"  <tr>\n    <th>{key}</th>\n    <td>"

            if key in ["entities", "data_insights"]:
                html += "<ul>\n"
                for item in value.split("\n- "):
                    if item.strip():
                        html += f"      <li>{item.strip()}</li>\n"
                html += "    </ul>"
            elif key == "hypothetical_questions":
                html += "<ol>\n"
                for item in value.split("\n"):
                    if item.strip():
                        html += f"      <li>{item.strip()}</li>\n"
                html += "    </ol>"
            else:
                html += value

            html += "</td>\n  </tr>\n"

    html += "</table>"
    return html

In [ ]:
print(table_summary_data_list[1])

In [ ]:
extract_table = extract_include_tag_content(table_summary_data_list[1], "table")
print(extract_table)

In [ ]:
markdown_table = convert_to_markdown_table(extract_table)
print(markdown_table)

Markdown의 제목을 기준으로 Split

In [ ]:
from langchain.text_splitter import MarkdownHeaderTextSplitter # 마크다운 헤더를 기준으로 텍스트를 분할하는 클래스입니다.

def markdown_header_text_splitter(stripheaders=False):
    # 마크다운 헤더 레벨에 따라 분할할 기준을 정의합니다.
    headers_to_split_on = [
        ("#", "Header 1"),
        ("##", "Header 2"),
        ("###", "Header 3"),
        ("####", "Header 4"),
        ("#####", "Header 5"),
        ("######", "Header 6"),  
        ("#######", "Header 7"), 
        ("########", "Header 8")
    ]
    return MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on, strip_headers=stripheaders) # 텍스트 분할기를 초기화합니다.

docs_string = page_content_list[1] # 첫 번째 문서의 내용을 문자열로 저장합니다.
docs_result = markdown_header_text_splitter().split_text(docs_string) # 텍스트를 헤더 기준으로 분할합니다.

print("Length of splits: " + str(len(docs_result))) # 분할된 청크의 개수를 출력합니다.
docs_result

벡터 DB 저장 형식으로 문서 구조 변환 및 생성

In [ ]:
def create_document(content, metadata):
    """
    문서 객체를 생성합니다.

    Args:
        content (str): 문서의 내용
        metadata (dict): 문서의 메타데이터

    Returns:
        Document: 생성된 문서 객체
    """
    return Document(page_content=content, metadata=metadata)


def process_image_element(image_cropped_item, image_summary_list, page_number):
    """
    이미지 요소를 처리합니다.

    Args:
        element (dict): 이미지 요소 정보
        state (dict): 현재 상태
        page_number (str): 페이지 번호

    Returns:
        tuple: 마크다운 문자열과 문서 객체 리스트
    """
    image_id = image_cropped_item["id"]
    image_summary = image_summary_list[image_id]
    image_path = image_cropped_item["image"]
    image_path_md = f"![{image_path}]({image_path})"

    image_summary = extract_include_tag_content(image_summary, "image") # LLM에서 추가된 주석 제거

    # image_summary_md = convert_to_markdown_table(image_summary)    
    markdown = f"{image_path_md}"

    image_summary_clean = extract_non_tag_content(
        image_summary, "hypothetical_questions"
    )

    docs = [
        create_document(
            image_summary_clean,
            {
                "type": "image",
                "image": image_path,
                "page": page_number,
                "source": _filePath,
                # "id": image_id, # id가 중복될 경우 azure ai search에 저장 시 누락됨
                "doc_name": _file_name,
            },
        )
    ]

    hypo_docs = []

    hypothetical_questions = extract_tag_content(
        image_summary, "hypothetical_questions"
    )
    if hypothetical_questions != None:
        hypo_docs.append(
            create_document(
                hypothetical_questions,
                {
                    "type": "hypothetical_questions",
                    "image": image_path,
                    "summary": image_summary_clean,
                    "page": page_number,
                    "source": _filePath,
                    # "id": image_id,
                    "doc_name": _file_name,
                },
            )
        )

    return markdown, docs, hypo_docs


def process_table_element(table_cropped_item, table_summary_list, page_number):
    """
    테이블 요소를 처리합니다.

    Args:
        element (dict): 테이블 요소 정보
        state (dict): 현재 상태
        page_number (str): 페이지 번호

    Returns:
        tuple: 마크다운 문자열과 문서 객체
    """
    table_id = table_cropped_item["id"]
    table_summary = table_summary_list[table_id]
    table_markdown = table_cropped_item["markdown"]
    table_path = table_cropped_item["image"]
    table_path_md = f"![{table_path}]({table_path})"

    table_summary = extract_include_tag_content(table_summary, "table") # LLM에서 추가된 주석 제거

    # table_summary_md = convert_to_markdown_table(table_summary)
    markdown = f"{table_path_md}\n{table_markdown}"

    table_summary_clean = extract_non_tag_content(
        table_summary, "hypothetical_questions"
    )

    docs = [
        create_document(
            table_summary_clean,
            {
                "type": "table",
                "table": table_path,
                "markdown": table_markdown,
                "page": page_number,
                "source": _filePath,
                # "id": table_id,
                "doc_name": _file_name,
            },
        )
    ]

    hypo_docs = []

    hypothetical_questions = extract_tag_content(
        table_summary, "hypothetical_questions"
    )
    if hypothetical_questions != None:
        hypo_docs.append(
            create_document(
                hypothetical_questions,
                {
                    "type": "hypothetical_questions",
                    "table": table_path,
                    "summary": table_summary_clean,
                    "markdown": table_markdown,
                    "page": page_number,
                    "source": _filePath,
                    # "id": table_id,
                    "doc_name": _file_name,
                },
            )
        )

    return markdown, docs, hypo_docs


def process_text_element(page):
    """
    텍스트 요소를 처리합니다.

    Args:
        element (dict): 텍스트 요소 정보

    Returns:
        str: 텍스트 내용
    """
    return page


def process_page(page, page_number, summarylist, image_cropped_list, table_cropped_list, image_summary_data_list, table_summary_data_list, text_splitter):
    """
    페이지를 처리합니다.

    Args:
        page (dict): 페이지 정보
        state (dict): 현재 상태
        page_number (str): 페이지 번호
        text_splitter (RecursiveCharacterTextSplitter): 텍스트 분할기

    Returns:
        tuple: 마크다운 문자열 리스트와 문서 객체 리스트
    """
    markdowns = []
    docs = []
    hypo_docs = []
    page_texts = []

    imageinfo_page_items = [imageinfo for imageinfo in image_cropped_list if imageinfo["page_number"] == page_number]
    for imageitem in imageinfo_page_items:
        markdown, element_docs, hypo_doc = process_image_element(
            imageitem, image_summary_data_list, page_number
        )
        markdowns.append(markdown)
        docs.extend(element_docs)
        hypo_docs.extend(hypo_doc)

    tableinfo_page_items = [tableinfo for tableinfo in table_cropped_list if tableinfo["page_number"] == page_number]
    for tableitem in tableinfo_page_items:
        markdown, element_docs, hypo_doc = process_table_element(
            tableitem, table_summary_data_list, page_number
        )
        markdowns.append(markdown)
        docs.extend(element_docs)
        hypo_docs.extend(hypo_doc)

    text = process_text_element(page)
    markdowns.append(text)
    page_texts.append(text)

    page_text = "\n".join(page_texts)
    split_texts = text_splitter.split_text(page_text)

    text_summary = summarylist[page_number]
    docs.append(
        create_document(
            text_summary,
            metadata={
                "type": "page_summary",
                "page": page_number,
                "source": _filePath,
                "text": page_text,
                "doc_name": _file_name,
            },
        )
    )

    for text in split_texts:
        docs.append(
            create_document(
                text.page_content,
                metadata={
                    "type": "text",
                    "page": page_number,
                    "source": _filePath,
                    "summary": text_summary,
                    "doc_name": _file_name,
                },
            )
        )

    return markdowns, docs, hypo_docs


def process_document(pageContentList, pageSummaryList, imageCroppedList, tableCroppedList, imageSummaryList, tableSummaryList):
    """
    전체 문서를 처리합니다.

    Args:
        state (dict): 현재 상태

    Returns:
        tuple: 마크다운 문자열 리스트와 문서 객체 리스트
    """
    markdowns = []
    docs = []
    hypo_docs = []
    # text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    markdown_splitter = markdown_header_text_splitter()

    for page_number, page in pageContentList.items():
        # print(f"{page_number, page}")
        page_markdowns, page_docs, page_hypo_docs = process_page(
            page, 
            page_number, 
            pageSummaryList, 
            imageCroppedList, 
            tableCroppedList, 
            imageSummaryList, 
            tableSummaryList, 
            markdown_splitter,
        )
        markdowns.extend(page_markdowns)
        docs.extend(page_docs)
        hypo_docs.extend(page_hypo_docs)

    return markdowns, docs, hypo_docs

In [ ]:
markdowns, docs, hypo_docs = process_document(
    page_content_list, 
    summary_list,
    image_cropped_list,
    table_cropped_list,
    image_summary_data_list,
    table_summary_data_list)

In [ ]:
# Markdown 파일로 텍스트 저장
with open(_filePath.replace(".pdf", ".md"), "w", encoding="utf-8") as f:
    f.write("\n\n".join(markdowns))

print(f"텍스트가 '{_filePath.replace('.pdf', '.md')}' 파일로 저장되었습니다.")

In [ ]:
len(docs)

In [ ]:
print(docs[0])

In [ ]:
len(hypo_docs)

In [ ]:
print(hypo_docs[0])

벡터 DB에 저장할 문서 범위 설정

In [ ]:
all_docs = docs + hypo_docs
len(all_docs)

In [ ]:
# 모든 데이터가 string 타입이 아닐 경우 azure ai search에 저장 시 타입 오류가 발생
for doc in all_docs:
    # 모든 메타데이터 필드를 문자열로 변환
    for key, value in doc.metadata.items():
        if not isinstance(value, str):
            doc.metadata[key] = str(value)

In [ ]:
for i, d in enumerate(all_docs):
    print(i, d.metadata["type"])

In [ ]:
print(all_docs[6])

In [ ]:
print(all_docs[7])

In [ ]:
print(all_docs[10])

In [ ]:
print(all_docs[58])

임베딩 모델 생성

In [ ]:
from langchain_openai import AzureOpenAIEmbeddings
from langchain_community.vectorstores.azuresearch import AzureSearch # AzureSearch 벡터 저장소 클래스를 임포트합니다.
from azure.search.documents.indexes.models import (
    ScoringProfile,
    SearchableField,
    SearchField,
    SearchFieldDataType,
    SimpleField,
    TextWeights
) # Azure Search 인덱스 모델 클래스를 임포트합니다.

embedding = AzureOpenAIEmbeddings(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"), # Azure OpenAI API 키를 환경 변수에서 가져옵니다.
    azure_deployment=os.getenv('AZURE_OPENAI_ADA_EMBEDDING_DEPLOYMENT_NAME'), # 사용할 Azure 배포 모델을 설정합니다.
    openai_api_version=os.getenv('AZURE_OPENAI_ADA_DEPLOYMENT_VERSION'), # OpenAI API 버전을 설정합니다.
    azure_endpoint =os.getenv('AZURE_OPENAI_ENDPOINT') # Azure OpenAI 엔드포인트를 환경 변수에서 가져옵니다.
)  # OpenAI 임베딩을 사용합니다.

embedding_function = embedding.embed_query # 임베딩 함수 설정

azure ai search 인덱스 필드 구성

In [ ]:
fields = [
    # 문서의 고유 ID 필드 정의
    SimpleField(
        name="id",
        type=SearchFieldDataType.String,
        key=True,
        filterable=True,
    ),
    # 검색 가능한 콘텐츠 필드 정의
    SearchableField(
        name="content",
        type=SearchFieldDataType.String,
        searchable=True,
        analyzer_name='ko.microsoft',
        #search_analyzer_name='ko.microsoft',
    ),
    # 콘텐츠 벡터 필드 정의
    SearchField(
        name="content_vector",
        type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
        searchable=True,
        vector_search_dimensions=len(embedding_function("Text")), # 임베딩 벡터의 차원 수 설정
        vector_search_profile_name="myHnswProfile", # 벡터 검색 프로파일 이름 설정
    ),
    # 메타데이터 필드 정의
    SearchableField(
        name="metadata",
        type=SearchFieldDataType.String,
        searchable=True,
        analyzer_name='ko.microsoft',
    ),
    # 메타데이터 벡터 필드 정의
    SearchField(
        name="metadata_vector",
        type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
        searchable=True,
        vector_search_dimensions=len(embedding_function("Text")), # 임베딩 벡터의 차원 수 설정
        vector_search_profile_name="myHnswProfile", # 벡터 검색 프로파일 이름 설정
    ),
    # 파일명을 저장하기 위한 필드
    SearchableField(
        name="doc_name",
        type=SearchFieldDataType.String,
        searchable=True,
        filterable=True,        
        analyzer_name='ko.microsoft',
    ),
    # 파일 경로를 저장하기 위한 필드
    SearchableField(
        name="source",
        type=SearchFieldDataType.String,
        searchable=True,
        analyzer_name='ko.microsoft',
    ),
    # 이미지 경로를 저장하기 위한 필드
    SearchableField(
        name="image",
        type=SearchFieldDataType.String,
        searchable=True,
        analyzer_name='ko.microsoft',
    ),
]

azure ai search 인덱스 생성

In [ ]:
# AzureSearch 객체를 생성하여 다중 모달 벡터 저장소를 초기화합니다.
vector_store_multi_modal: AzureSearch = AzureSearch(
    azure_search_endpoint=os.getenv('AZURE_SEARCH_ENDPOINT'), # Azure Search 엔드포인트 설정
    azure_search_key=os.getenv('AZURE_SEARCH_API_KEY'), # Azure Search 키 설정
    index_name=_index_name, # 인덱스 이름 설정
    embedding_function=embedding_function, # 임베딩 함수 설정
    semantic_configuration_name='default',
    fields=fields, # 인덱스 필드 설정
    # **kwargs
)

azure ai search 인덱스에 문서 추가

In [ ]:
vector_store_multi_modal.add_documents(documents=all_docs) # 문서 리스트를 Azure Search 인덱스에 추가합니다.


Retriever 생성

In [ ]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever

# bm25 retriever와 faiss retriever를 초기화합니다.
bm25_retriever = BM25Retriever.from_documents(
    all_docs,
)
bm25_retriever.k = 5  # BM25Retriever의 검색 결과 개수를 1로 설정합니다.

In [ ]:
# Azure ai search retriever 생성
# vector_retriever = vector_store_multi_modal.as_retriever(search_kwargs={"k": 5})
vector_retriever = vector_store_multi_modal.as_retriever(k=5)

In [ ]:
# 앙상블 retriever를 초기화합니다.
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, vector_retriever],
    weights=[0.7, 0.3],
)

Relevance Checker 로직을 활용한 중요 정보 필터링

In [ ]:
from pydantic import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate

# 데이터 모델
class GradeRetrievalQuestion(BaseModel):
    """A binary score to determine the relevance of the retrieved documents to the question."""

    score: str = Field(
        description="Whether the retrieved context is relevant to the question, 'yes' or 'no'"
    )

# 데이터 모델
class GradeRetrievalAnswer(BaseModel):
    """A binary score to determine the relevance of the retrieved documents to the answer."""

    score: str = Field(
        description="Whether the retrieved context is relevant to the answer, 'yes' or 'no'"
    )


"""
당신은 {target_variable}와(과) 관련된 검색된 문서의 적합성을 평가하는 평가자입니다. 
이는 엄격한 검사가 필요하지 않습니다. 
목표는 잘못된 검색 결과를 걸러내는 것입니다. 
문서에 {target_variable}와(과) 관련된 키워드나 의미가 포함되어 있다면, 이를 적합한 것으로 평가하세요. 
문서가 {target_variable}와(과) 관련이 있는지 여부를 나타내기 위해 '예' 또는 '아니오'로 이진 점수를 부여하세요.
"""
def azure_openai_relevance_grader(target="retrieval-question"):
    llm = llm_gpt4o_mini()

    if target == "retrieval-question":
        structured_llm_grader = llm.with_structured_output(
            GradeRetrievalQuestion
        )
    elif target == "retrieval-answer":
        structured_llm_grader = llm.with_structured_output(
            GradeRetrievalAnswer
        )
    else:
        raise ValueError(f"Invalid target: {target}")
    
    # 프롬프트
    target_variable = (
        "user question" if target == "retrieval-question" else "answer"
    )
    system = f"""You are a grader assessing relevance of a retrieved document to a {target_variable}. \n 
        It does not need to be a stringent test. The goal is to filter out erroneous retrievals. \n
        If the document contains keyword(s) or semantic meaning related to the {target_variable}, grade it as relevant. \n
        Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to {target_variable}."""

    grade_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system),
            (
                "human",
                f"Retrieved document: \n\n {{context}} \n\n {target_variable}: {{input}}",
            ),
        ]
    )
    grader_prompt = grade_prompt

    retrieval_grader_oai = grader_prompt | structured_llm_grader
    return retrieval_grader_oai

In [ ]:
# Groundness Checker 생성
groundedness_check = azure_openai_relevance_grader()

In [ ]:
retrieved_documents = ensemble_retriever.invoke(
    "K팝 경제를 설명해줘"
)
retrieved_documents

In [ ]:
def clean_retrieved_documents(retrieved_documents):
    clean_docs = []

    for doc in retrieved_documents:
        metadata = doc.metadata
        new_metadata = {}
        content = doc.page_content

        # 문서 타입이 'page_summary' 또는 'text'인 경우
        if metadata["type"] in ["page_summary", "text"]:
            # 페이지 번호와 소스 정보를 새 메타데이터에 추가
            if "page" in metadata:
                new_metadata["page"] = metadata["page"]
            if "source" in metadata:
                new_metadata["source"] = metadata["source"]
            # 'text' 타입인 경우 요약 정보도 추가
            if metadata["type"] == "text":
                # content += f'\n\n<summary>{metadata["summary"]}</summary>'
                new_metadata["summary"] = metadata["summary"]
            clean_docs.append(Document(page_content=content, metadata=new_metadata))

        # 문서 타입이 'image'인 경우
        elif metadata["type"] == "image":
            image_path = metadata["image"]
            # 페이지 번호와 소스 정보를 새 메타데이터에 추가
            if "page" in metadata:
                new_metadata["page"] = metadata["page"]
            if "source" in metadata:
                new_metadata["source"] = metadata["source"]
            # 내용을 마크다운 테이블 형식으로 변환
            # content = content
            content = convert_to_markdown_table(content)

            clean_docs.append(Document(page_content=content, metadata=new_metadata))

        # 문서 타입이 'table'인 경우
        elif metadata["type"] == "table":
            table_path = metadata["table"]
            table_markdown = metadata["markdown"]
            # 페이지 번호와 소스 정보를 새 메타데이터에 추가
            if "page" in metadata:
                new_metadata["page"] = metadata["page"]
            if "source" in metadata:
                new_metadata["source"] = metadata["source"]
            # 내용을 마크다운 테이블 형식으로 변환하고 원본 마크다운과 결합
            # content = f"{content}\n\n{table_markdown}"
            content = f"{convert_to_markdown_table(content)}\n\n{table_markdown}"

            clean_docs.append(Document(page_content=content, metadata=new_metadata))

        # 문서 타입이 'hypothetical_questions'인 경우
        elif metadata["type"] == "hypothetical_questions":
            # 내용을 요약 정보로 대체
            content = metadata["summary"]
            # 페이지 번호와 소스 정보를 새 메타데이터에 추가
            if "page" in metadata:
                new_metadata["page"] = metadata["page"]
            if "source" in metadata:
                new_metadata["source"] = metadata["source"]
            clean_docs.append(Document(page_content=content, metadata=new_metadata))

    return clean_docs

In [ ]:
# 함수 사용 예시
# 앙상블 리트리버를 사용하여 질문에 대한 문서 검색
retrieved_documents = ensemble_retriever.invoke(
    "K팝 경제를 설명해줘"
)
# 검색된 문서를 정제하여 깨끗한 형태로 변환
cleaned_documents = clean_retrieved_documents(retrieved_documents)

In [ ]:
for doc in cleaned_documents:
    print(doc.page_content)
    print("---" * 30)
    print(doc.metadata)
    print("===" * 30, end="\n\n\n")

In [ ]:
def retrieve_and_check(question, use_checker=True):
    # 질문에 대한 문서를 검색합니다.
    retrieved_documents = ensemble_retriever.invoke(question)

    # 검색된 문서를 정제합니다.
    cleaned_documents = clean_retrieved_documents(retrieved_documents)

    filtered_documents = []
    if use_checker:
        # 검사기를 사용하는 경우, 각 문서의 내용과 질문을 입력으로 준비합니다.
        checking_inputs = [
            {"context": doc.page_content, "input": question}
            for doc in cleaned_documents
        ]

        # 준비된 입력을 사용하여 일괄 검사를 수행합니다.
        checked_results = groundedness_check.batch(checking_inputs)

        # 검사 결과가 'yes'인 문서만 필터링합니다.
        filtered_documents = [
            doc
            for doc, result in zip(cleaned_documents, checked_results)
            if result.score == "yes"
        ]
    else:
        # 검사기를 사용하지 않는 경우, 모든 정제된 문서를 그대로 사용합니다.
        filtered_documents = cleaned_documents

    # 필터링된 문서를 반환합니다.
    return filtered_documents

In [ ]:
retrieve_and_check("K팝 경제를 설명해줘")

LLM 답변 생성

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.prompts import PromptTemplate

# 프롬프트 생성(Create Prompt)
# 프롬프트를 생성합니다.

"""
당신은 질문에 답하는 작업을 지원하는 도우미입니다.
다음 제공된 검색된 컨텍스트를 사용하여 질문에 답하세요.
답을 모를 경우, "모르겠습니다"라고 답하세요.
답변은 한국어로 작성해야 합니다.

지침:
1. 질문의 의도를 이해하고 가장 적절한 답변을 제공하세요.
2. 질문의 맥락과 질문자가 왜 이를 물었는지 스스로에게 물어보고, 이를 기반으로 적절한 답변을 작성하세요.
3. 검색된 컨텍스트 중에서 질문과 직접적으로 관련된 가장 중요한 내용을 선택하여 답변을 작성하세요.
4. 간결하고 논리적인 답변을 만드세요. 답변을 작성할 때, 단순히 선택된 내용을 나열하지 말고, 문맥에 맞게 자연스럽게 문단으로 재구성하세요.
5. 질문에 대한 컨텍스트를 검색하지 않았거나, 검색된 문서의 내용이 질문과 관련이 없을 경우, "제가 가진 자료에서는 해당 질문에 대한 답을 찾을 수 없습니다."라고 답하세요.
6. 답변은 주요 내용을 요약한 표 형식으로 작성하세요.
7. 답변에는 모든 출처와 페이지 번호를 포함해야 합니다.
8. 답변은 최대한 상세하게 작성해야 합니다.
9. 답변은 "📚 문서에서 검색한 내용기반 답변입니다"로 시작하고, "📌 출처"로 끝내야 합니다.
10. 페이지 번호는 정수로 표기해야 합니다.
컨텍스트:
{context}

예시 형식:
📚 문서에서 검색한 내용기반 답변입니다

(답변의 간단한 요약)
(질문과 관련된 문맥에 표가 포함되어 있을 경우 표를 포함)
(질문과 관련된 문맥에 이미지 설명이 있을 경우 설명 포함)
(질문에 대한 상세한 답변)

📌 출처
[여기에 파일명(.pdf만 허용)과 페이지 번호를 작성합니다.]

파일명.pdf, 192쪽
파일명.pdf, 192쪽
"""

prompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Answer in Korean.

# Direction:
Make sure you understand the intent of the question and provide the most appropriate answer.
1. Ask yourself the context of the question and why the questioner asked it, think about the question, and provide an appropriate answer based on your understanding.
2. Select the most relevant content (the key content that directly relates to the question) from the context in which it was retrieved to write your answer.
3. Create a concise and logical answer. When creating your answer, don't just list your selections, but rearrange them to fit the context so they flow naturally into paragraphs.
4. If you haven't searched for context for the question, or if you've searched for a document but its content isn't relevant to the question, you should say ‘I can't find an answer to that question in the materials I have’.
5. Write your answer in a table of key points.
6. Your answer must include all sources and page numbers.
7. Your answer must be written in Korean.
8. Be as detailed as possible in your answer.
9. Begin your answer with ‘This answer is based on content found in the document **📚’ and end with ‘**📌 source**’.
10. Page numbers should be whole numbers.

#Context: 
{context}

###

#Example Format:
**📚 문서에서 검색한 내용기반 답변입니다**

(brief summary of the answer)
(include table if there is a table in the context related to the question)
(include image explanation if there is a image in the context related to the question)
(detailed answer to the question)

**📌 출처**
[here you only write filename(.pdf only), page]

- 파일명.pdf, 192쪽
- 파일명.pdf, 192쪽
- ...

###

#Question:
{question}

#Answer:"""
)

# 단계 7: 언어모델(LLM) 생성
# 모델(LLM) 을 생성합니다.
llm = llm_gpt4o()

# 단계 8: 체인(Chain) 생성
chain = (
    {"context": RunnableLambda(retrieve_and_check), "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
print(chain.invoke("K팝 경제를 설명해줘"))